# Data quality analysis for `product_specifications` table

This table holds detailed information about the product like brand, type, color and a description. As all other tables, it repeats the columns from table `default_product_info`, that could be safely removed, keeping only the identifier columns (`aw_product_id` and `merchant_product_id`) to allow future joins.

In [0]:
import pyspark.sql.functions as F
import plotly.express as px
import plotly.graph_objects as go

In [0]:
%sql
DESCRIBE mvp.bronze_feeds.product_specifications

col_name,data_type,comment
aw_deep_link,string,The unique Awin tracking link for the product.
product_name,string,The full title or name of the product.
aw_product_id,bigint,Awin's unique internal identifier for the product.
merchant_product_id,string,The unique ID assigned to the product by the merchant.
merchant_image_url,string,Direct URL to the product image on the merchant's server.
description,string,Detailed description of the product features.
merchant_category,string,The category name assigned by the merchant.
search_price,double,The current selling price used for search filtering.
brand_name,string,The manufacturer or brand name.
brand_id,string,Unique identifier for the brand.


The columns without description are the same from `default_product_info`, let's keep only the identifiers from these.

In [0]:
columns_to_keep = [
  "aw_product_id",
  "merchant_product_id",

  "brand_name",

  "brand_id",
  "colour",
  
  "condition",
  "product_model",
  "dimensions",
  "product_type",

  "product_short_description",
  "specifications"
]

silver_product_specifications = \
    spark.read.table("mvp.bronze_feeds.product_specifications")\
      .select(*columns_to_keep)
display(silver_product_specifications.limit(10))

aw_product_id,merchant_product_id,brand_name,brand_id,colour,condition,product_model,dimensions,product_type,product_short_description,specifications
42205909314,275877880,ON Running,24825,Cinza,null,null,null,null,null,null
42205908872,275877881,ON Running,24825,Cinza,null,null,null,null,null,null
42205909434,275877882,ON Running,24825,Cinza,null,null,null,null,null,null
42205909548,275877883,ON Running,24825,Cinza,null,null,null,null,null,null
42205907626,275877884,ON Running,24825,Cinza,null,null,null,null,null,null
42205907978,275877885,ON Running,24825,Cinza,null,null,null,null,null,null
42205909326,275877886,ON Running,24825,Cinza,null,null,null,null,null,null
42205907629,275878129,ON Running,24825,Bege,null,null,null,null,null,null
42205908914,275878131,ON Running,24825,Bege,null,null,null,null,null,null
42205908687,275878133,ON Running,24825,Bege,null,null,null,null,null,null


All fields are categorical, let's start by checking how many different values we have for each of them.

In [0]:
cols = [c for c in columns_to_keep if c not in ["aw_product_id", "merchant_product_id"]]

stats = []
df = silver_product_specifications

for col in cols:
    null_count = df.filter(F.col(col).isNull()).count()
    total = df.count()
    null_pct = (null_count / total) * 100 if total > 0 else 0
    stats.append({
        "column": col,
        "null_count": null_count,
        "total": total,
        "null_pct": null_pct
    })

import pandas as pd

stats_df = pd.DataFrame(stats)

fig = go.Figure()

fig.add_bar(
    x=stats_df["column"],
    y=stats_df["null_pct"],
    text=[f"{col}<br>{pct:.1f}%" for col, pct in zip(stats_df["column"], stats_df["null_pct"])],
    textposition="outside",
    name="Null Percentage"
)

fig.update_layout(
    title="Columns null percentage",
    xaxis_title="Column",
    yaxis_title="Null Percentage (%)",
    showlegend=False,
    height=700,
    width=1100,
    margin=dict(l=120, r=40, t=80, b=180),
    xaxis=dict(
        tickmode='array',
        tickvals=list(stats_df["column"]),
        ticktext=list(stats_df["column"]),
        tickangle=45
    )
)

fig.update_traces(marker_line_width=1.5)

display(fig)

From this chart, it's clear that only a few columns are useful as a lot of them have too much null values. Let's keep only columns `brand_name`, `colour` and `condition`.

In [0]:
silver_product_specifications = silver_product_specifications.select("brand_name", "colour", "condition")

display(silver_product_specifications.limit(10))

brand_name,colour,condition
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Bege,null
ON Running,Bege,null
ON Running,Bege,null


Let's keep the null values for `brand_name` and `colour`, but let's analyze values for condition.

In [0]:
condition_counts = silver_product_specifications.groupBy("condition").count().toPandas()
condition_counts["percentage"] = (condition_counts["count"] / condition_counts["count"].sum()) * 100

fig = px.pie(
    condition_counts,
    names="condition",
    values="count",
    title="Distinct Values for Condition",
    hole=0.3
)

fig.update_traces(
    textinfo='label+percent',
    hovertemplate='%{label}: %{value} (%{percent:.1%})'
)

fig.update_layout(
    legend_title_text="Condition (Percentage)",
    legend=dict(
        itemsizing='constant',
        font=dict(size=14)
    )
)

display(fig)

There is a value `99.9`, with a very small participation, that clearly comes from an error, let's remove these rows.

Also, we have a small set of values with null, that means, we don't know if the product is new or used for these, let's just keep the null value.

In [0]:
silver_product_specifications = silver_product_specifications\
        .where(F.col("condition").isNull() | (F.lower(F.col("condition")) == F.lit("new")) | (F.lower(F.col("condition")) == F.lit("used")))

display(silver_product_specifications)

brand_name,colour,condition
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Cinza,null
ON Running,Bege,null
ON Running,Bege,null
ON Running,Bege,null


Let's normalize the fields `brand_name` and `colour` by lowecasing the string and removing the leading and trailing spaces. After this, let's see the availabe values.

In [0]:
silver_product_specifications = silver_product_specifications\
    .withColumn("brand_name", F.trim(F.lower(F.col("brand_name"))))\
    .withColumn("colour", F.trim(F.lower(F.col("colour"))))

display(silver_product_specifications.limit(10))

brand_name,colour,condition
on running,cinza,null
on running,cinza,null
on running,cinza,null
on running,cinza,null
on running,cinza,null
on running,cinza,null
on running,cinza,null
on running,bege,null
on running,bege,null
on running,bege,null


In [0]:
top_brands = silver_product_specifications.groupBy("brand_name")\
    .count()\
    .orderBy(F.desc("count"))\
    .limit(100)\
    .toPandas()

fig = px.bar(
    top_brands,
    x="brand_name",
    y="count",
    title="Top 100 Brand Names by Row Count",
    log_y=True
)

fig.update_layout(
    xaxis_title="Brand Name",
    yaxis_title="Row Count (Log Scale)",
    xaxis_tickangle=45,
    height=700,
    width=1100,
    margin=dict(l=120, r=40, t=80, b=180)
)

display(fig)

In [0]:
top_colours = silver_product_specifications.groupBy("colour")\
    .count()\
    .orderBy(F.desc("count"))\
    .limit(100)\
    .toPandas()

fig = px.bar(
    top_colours,
    x="colour",
    y="count",
    title="Top 100 Colours by Row Count",
    log_y=True
)

fig.update_layout(
    xaxis_title="Colour",
    yaxis_title="Row Count (Log Scale)",
    xaxis_tickangle=45,
    height=700,
    width=1100,
    margin=dict(l=120, r=40, t=80, b=180)
)

display(fig)

Agora, para fins de documentação, vamos anotar os valores possíveis das colunas.

In [0]:
silver_product_specifications_summary = silver_product_specifications.groupBy(
    F.col("brand_name").alias("brand_name_value"),
    F.col("colour").alias("color_value"),
    F.col("condition").alias("condition_value")
).count().withColumnRenamed("count", "rows_count")

display(silver_product_specifications_summary.limit(10))

brand_name_value,color_value,condition_value,rows_count
on running,cinza,null,60
on running,bege,null,42
on running,branco,null,148
on running,preto,null,197
on running,azul,null,43
asics,branco,null,20
asics,azul,null,52
olympikus,azul,null,17
new balance,preto,null,107
puma,preto,null,28


Desta forma, a transformação final que construirá a tabela na silver será.

In [0]:
columns_to_keep = [
  "aw_product_id",
  "merchant_product_id",

  "brand_name",
  "colour",
  
  "condition"
]

silver_product_specifications = \
    spark.read.table("mvp.bronze_feeds.product_specifications")\
      .select(*columns_to_keep)

silver_product_specifications = silver_product_specifications\
    .where(F.col("condition").isNull() | (F.lower(F.col("condition")) == F.lit("new")) | (F.lower(F.col("condition")) == F.lit("used")))\
    .withColumn("brand_name", F.trim(F.lower(F.col("brand_name"))))\
    .withColumn("colour", F.trim(F.lower(F.col("colour"))))\
    .withColumn("condition", F.trim(F.lower(F.col("condition"))))

display(silver_product_specifications.limit(10))

aw_product_id,merchant_product_id,brand_name,colour,condition
42205909314,275877880,on running,cinza,null
42205908872,275877881,on running,cinza,null
42205909434,275877882,on running,cinza,null
42205909548,275877883,on running,cinza,null
42205907626,275877884,on running,cinza,null
42205907978,275877885,on running,cinza,null
42205909326,275877886,on running,cinza,null
42205907629,275878129,on running,bege,null
42205908914,275878131,on running,bege,null
42205908687,275878133,on running,bege,null


- Let's do this transformation as a task `product_specifications_silver` after the task `product_specifications_bronze`.